# Функции для тестового задания

In [18]:
import pandas as pd
import os
from os import walk
import datetime
pd.set_option('display.max_columns', 500)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date

import csv

import folium
from folium import plugins
from folium.plugins import HeatMap
import branca
import branca.colormap as cm
from datetime import timedelta

# Графики
`show_chart` отображает изменение координат широты и долготы по времени

In [19]:
def show_chart(fig, data_a, data_b ):
    """
    The function draws 2 chatrs: msisdn lattitude on time and msisdn longitude on time

"""

    data_a_copy = data_a
    data_b_copy = data_b

    plt.subplot(1, 2, 1)
    feature = 'long'
    plt.scatter(data_a_copy['datetime'], data_a_copy[feature], alpha = .5, c = 'blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_max'],'-o',alpha = .3, color='blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_min'],'-o',alpha = .3, color='blue')
    plt.scatter(data_b_copy['datetime'], data_b_copy[feature], alpha = .5, c = 'red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_max'],'-o',alpha = .3, color='red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_min'],'-o',alpha = .3, color='red')
    plt.xlim(pd.concat([data_a,data_b])['datetime'].min(), pd.concat([data_a,data_b])['datetime'].max())
    plt.xticks(rotation=70)

    plt.subplot(1, 2, 2)
    feature = 'lat'
    plt.scatter(data_a_copy['datetime'], data_a_copy[feature], alpha = .5, c = 'blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_max'],'-o',alpha = .3, color='blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_min'],'-o',alpha = .3, color='blue')
    plt.scatter(data_b_copy['datetime'], data_b_copy[feature], alpha = .5, c = 'red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_max'],'-o',alpha = .3, color='red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_min'],'-o',alpha = .3, color='red')
    plt.xlim(pd.concat([data_a,data_b])['datetime'].min(), pd.concat([data_a,data_b])['datetime'].max())
    plt.xticks(rotation=70)
    
    plt.show()
    return fig

`show_circles_on_map` отображает точки в которых был абонент на карте

In [20]:
def show_circles_on_map(m, data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())

    for _, row in data.iterrows():
        folium.Circle(
            radius=100,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True,
            fill_opacity = .5
        ).add_to(m)

    return m

`haversine_array` определяет расстояние в метрах по координатам

In [21]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

## Обработка координат

`BS_rectangle_coordinate` по координатам БС, максимальной дистанции приема и направлению и углу покрятия определяет координаты прямоугольника, в котором мог находиться абонент при регистрации на БС

In [22]:
def BS_rectangle_coordinate(df_, CENTER_LAT, CENTER_LONG):
    df = df_.copy()
    lat_length = haversine_array(CENTER_LAT -.5, CENTER_LONG, CENTER_LAT + .5, CENTER_LONG)
    long_length = haversine_array(CENTER_LAT, CENTER_LONG -.5, CENTER_LAT, CENTER_LONG + .5)
    max_dist_mean = df['max_dist'].mean()

    df['max_dist_correct'] = np.where(df['max_dist'] < 50, max_dist_mean, df['max_dist'])
    df['max_dist_correct']
    
    for feature in ['long', 'lat']:
        if (feature == 'long'):
            coord_length = long_length
        else:
            coord_length = lat_length
        df[feature + '_start'] = np.where(df['angle'] == 360,
                                 df[feature] - df['max_dist_correct']/coord_length/1000, df[feature] + df['max_dist_correct'] * np.where(feature == 'long', 
                                                                                           np.sin(df['start_angle']),
                                                                                           np.cos(df['start_angle'])
                                                                                                 )/coord_length/1000)

        df[feature + '_end'] = np.where(df['angle'] == 360,
                               df[feature]+df['max_dist_correct']/coord_length/1000,
                               df[feature] + df['max_dist_correct'] * np.where(feature == 'long', 
                                                                                           np.sin(df['end_angle']),
                                                                                           np.cos(df['end_angle'])
                                                                                                 )/coord_length/1000)
                                                                                         
        df[feature + '_min'] = df[[feature + '_start', feature + '_end', feature]].min(axis = 1)
        df[feature + '_max'] = df[[feature + '_start', feature + '_end', feature]].max(axis = 1)
        
        df[feature] = (df[feature + '_max'] + df[feature + '_min'])/2
        df.drop(columns = [feature + '_end', feature + '_start'], inplace = True)
    return df

`CreateTimeBeans` разбивает время на равные отрезки

In [23]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.datasets import make_blobs

def CreateTimeBeans(df_, n_bins_ = 10):
    df = df_.copy()
    
    dt_list = np.reshape(df.datetime.values.tolist(), (len(df.datetime.values.tolist()), 1))

    enc = KBinsDiscretizer(n_bins=n_bins_, encode='ordinal', strategy='uniform')
    dt_array_bins = enc.fit_transform(dt_list)

    df['datetime_bin'] = dt_array_bins[:,0]
    df['datetime_bin'] = df['datetime_bin'].astype('int')

    return df, df['datetime_bin'].unique()

`CreatePivotBase` создает промежуточный датафрейм, декартовое произведение интервалов времени и абонентов

In [24]:
def CreatePivotBase(df):
    
    df_data_msisdn = pd.DataFrame(df['msisdn'].drop_duplicates()).sort_values('msisdn')
    df_data_datetime = pd.DataFrame(df['datetime_bin'].drop_duplicates()).sort_values('datetime_bin')

    df_data_msisdn['K'] = 1
    df_data_datetime['K'] = 1
    df_data_pivot_base = df_data_msisdn.merge(df_data_datetime, on = ['K']).reset_index().drop(columns = ['index', 'K'])
    
    return df_data_pivot_base

`AggData` создает промежуточный датафрейм, декартовое произведение интервалов времени и абонентов

In [25]:
# аггрегация данных по интервалам
def AggData(df):
    #Создаем датафрейм в котором записаны часовые срезы расположения абонента и нумеруем в хронологическом порядке каждое состояние

    df_data_agg = df.groupby(['msisdn',  'datetime_bin'])['long_min', 'long_max', 'lat_min', 'lat_max', 'long', 'lat'].mean().reset_index()
    df_data_agg['datetime'] = df_data_agg['datetime_bin'] 
    df_data_agg = df_data_agg.sort_values(['msisdn', 'datetime_bin'])
    df_data_agg['state'] = df_data_agg.groupby(['msisdn'])['msisdn'].cumcount()
    return df_data_agg

`AggData` создает промежуточный датафрейм, декартовое произведение интервалов времени и абонентов

In [26]:
# cдвигаю событие на пол часа и объединяю с оригинальными событиями
def AddDataDelta(df_, delta = 30, n = 2):
    df = df_.copy()
    for i in range(0, n):
        df_delta = df_.copy()
        df_delta['datetime'] = df_delta['datetime'] - timedelta(seconds = (delta*60)) * (i + 1)
        df = df.append(df_delta)
    return df

`AggData` создает промежуточный датафрейм, декартовое произведение интервалов времени и абонентов

In [27]:
def LinearCombData(df_data_pivot_base, df_data_agg):
    
    df_data_pivot_base = df_data_pivot_base.merge(df_data_agg[['msisdn', 'datetime_bin', 'state']], on = ['msisdn', 'datetime_bin'], how = 'left')
    df_data_pivot_base['state_previous'] = 1- df_data_pivot_base['state'].isna()*1
    df_data_pivot_base['state_previous'] = df_data_pivot_base.groupby('msisdn')['state_previous'].cumsum() - 1
    
    df_data_pivot_base.drop(columns = ['state'], inplace = True)
    df_data_pivot_base['state_next'] = df_data_pivot_base['state_previous'] + 1

    for event in ['previous', 'next']:
        df_data_pivot_base = df_data_pivot_base.merge(df_data_agg[['msisdn', 'long', 'long_max', 'long_min', 'lat', 'lat_max', 'lat_min', 'datetime_bin', 'state']].rename(columns = {'long':'long_' + event, 'lat':'lat_' + event, 'long_max':'long_max_' + event,
                                                                             'lat_max':'lat_max_' + event,
                                                                             'long_min':'long_min_' + event,
                                                                             'lat_min':'lat_min_' + event,
                                                                              'datetime_bin':'datetime_bin_' + event,
                                                                              'state':'state_' + event}),
                                                                    on = ['msisdn', 'state_' + event],
                                                                    how = 'left')
    for feature in ['long', 'long_max', 'long_min', 'lat', 'lat_max', 'lat_min']:
    # расчитываю текущее событие для кажждого периода как линейную комбинацию окружающих реальных событий
        df_data_pivot_base[feature] = np.where(df_data_pivot_base[feature+'_previous'].isna(),
                                       df_data_pivot_base[feature+'_next'],
                                       np.where(df_data_pivot_base[feature+'_next'].isna(),
                                               df_data_pivot_base[feature+'_previous'],
                                               df_data_pivot_base[feature+'_previous'] + 
                                                    (df_data_pivot_base['datetime_bin'] - df_data_pivot_base['datetime_bin_previous'])/
                                                    (df_data_pivot_base['datetime_bin_next'] - df_data_pivot_base['datetime_bin_previous'])*
                                                    (df_data_pivot_base[feature+'_next'] - df_data_pivot_base[feature+'_previous'])))              
                                    

    return df_data_pivot_base[['msisdn', 'datetime_bin', 'long', 'long_max', 'long_min', 'lat', 'lat_max', 'lat_min']]

`AggData` создает промежуточный датафрейм, декартовое произведение интервалов времени и абонентов

In [28]:
def CombData(df_data_pivot_base, df_data_agg):
    
    df_data_pivot_base = df_data_pivot_base.merge(df_data_agg, on = ['msisdn', 'datetime_bin'], how = 'left')

    return df_data_pivot_base[['msisdn', 'datetime_bin', 'long', 'long_max', 'long_min', 'lat', 'lat_max', 'lat_min']]

`CreatePivot` Преобразует таблицу в массив с фичами

In [29]:
def CreatePivot(df_for_pivot):
    df_pivot = df_for_pivot.pivot(index = 'msisdn', columns = 'datetime_bin', values = 'long')
    df_pivot.columns = df_pivot.columns.astype('str') + '_' + 'long'

    for feature in ['long_max', 'long_min', 'lat', 'lat_max', 'lat_min']:
        df_agg_feature = df_for_pivot.pivot(index = 'msisdn', columns = 'datetime_bin', values = feature)
        df_agg_feature.columns = df_agg_feature.columns.astype('str') + '_' + feature
        df_pivot = pd.concat([df_pivot, df_agg_feature], axis = 1)

    return df_pivot.reset_index()

`CreateMsisdnPairs` создает таблицу с парами абонентов, на которых мы будем обучать модель

In [30]:
def CreateMsisdnPairs(train_pairs, df, train_random_frac_, random_random_frac):
    msisdn_pairs = pd.concat([train_pairs[[0, 1]].rename(columns = {0:'msisdn_x', 1:'msisdn_y'})
                         ], axis = 0).drop_duplicates()
    msisdn_pairs_dubli = pd.concat([train_pairs[[0, 1]].rename(columns = {0:'msisdn_x', 1:'msisdn_y'})
                         ,train_pairs[[1, 0]].rename(columns = {1:'msisdn_x', 0:'msisdn_y'})
                         ], axis = 0).drop_duplicates()
    
    msisdn_pairs['is_pair'] = 1
    msisdn_pairs['K'] = 1
    print('Sample pairs:', msisdn_pairs.shape[0])
    msisdn_pairs_dubli['K'] = 1
    
    df_data_msisdn = pd.DataFrame(df['msisdn'].drop_duplicates()).sort_values('msisdn')
    df_data_msisdn['K'] = 1

    # любой номер из пары с другими номерами не создает пары
    msisdn_pairs_part1 = df_data_msisdn[~df_data_msisdn['msisdn'].isin(msisdn_pairs_dubli['msisdn_x'])].rename(columns= {'msisdn':'msisdn_x'}).merge(msisdn_pairs_dubli[['K', 'msisdn_y']], on = ['K']).drop(columns = ['K'])
    msisdn_pairs_part1.drop_duplicates(inplace = True)
    msisdn_pairs_part1['is_pair'] = 0
    
    msisdn_pairs_part1 = msisdn_pairs_part1.sample(frac = train_random_frac_)
    print('Pair Random pairs:', msisdn_pairs_part1.shape[0])
    
    '''
    # пары, у которых оба номера не из списка
    msisdn_pairs_part2 = df_data_msisdn[~df_data_msisdn['msisdn'].isin(msisdn_pairs_dubli['msisdn_x'])].rename(columns= {'msisdn':'msisdn_x'}).merge(df_data_msisdn[~df_data_msisdn['msisdn'].isin(msisdn_pairs_dubli['msisdn_x'])].rename(columns= {'msisdn':'msisdn_y'}), on = ['K']).drop(columns = ['K'])
    print(msisdn_pairs_part2.columns)
    msisdn_pairs_part2.drop_duplicates(inplace = True)
    msisdn_pairs_part2['is_pair'] = 0
    msisdn_pairs_part2 = msisdn_pairs_part2[msisdn_pairs_part2['msisdn_x'] < msisdn_pairs_part2['msisdn_y']]
    msisdn_pairs_part2 = msisdn_pairs_part2.sample(frac = random_random_frac)
    print('Random Random pairs:', msisdn_pairs_part2.shape[0])
'''
    # объедтняю пары и не пары
    msisdn_pairs = pd.concat([msisdn_pairs, msisdn_pairs_part1
   #                       , msisdn_pairs_part2
                         ], axis = 0).drop(columns = ['K'])
    
    # перемешиваю первый и второй номер
    msisdn_pairs_half_1 = msisdn_pairs.sample(frac = .5)
    msisdn_pairs_half_2 = msisdn_pairs.append(msisdn_pairs_half_1).drop_duplicates(keep = False).rename(columns = {'msisdn_x':'msisdn_y', 'msisdn_y':'msisdn_x'})
    msisdn_pairs = msisdn_pairs_half_1.append(msisdn_pairs_half_2)
    print(msisdn_pairs.shape)
    
    return msisdn_pairs

`CreateAndProcessPivot` Обрабатываем pivot

In [32]:
def CreateAndProcessPivot(msisdn_pairs, data_agg_pivot, bins_list, identifier,
                          dist_ind = True,
                          coord_ind = True,
                          center_dist_ind = True,
                          CENTER_LAT = 55.7522200,
                          CENTER_LONG = 37.6155600):
    
    df_X = msisdn_pairs.merge(data_agg_pivot.rename(columns = {'msisdn':'msisdn_x'}), on = ['msisdn_x']).merge(data_agg_pivot.rename(columns = {'msisdn':'msisdn_y'}), on = ['msisdn_y'])
    # считаю расстояние в каждом часе
    columns_name_dist = list()
    columns_name_intersect = list()
    columns_name_lat_x = list()
    columns_name_long_x = list()
    columns_name_lat_y = list()
    columns_name_long_y = list()
    columns_name_center_dist_x = list()
    columns_name_center_dist_y = list()
    
    columns_name = list()
    columns_name_identifier = list()

    lat_length = haversine_array(CENTER_LAT -.5, CENTER_LONG, CENTER_LAT + .5, CENTER_LONG)
    long_length = haversine_array(CENTER_LAT, CENTER_LONG -.5, CENTER_LAT, CENTER_LONG + .5)

    for i in bins_list:
        str_i = str(int(i))
        columns_name_dist.append(str_i+"_dist")
        columns_name_intersect.append(str_i+"_intersect")
        columns_name_lat_x.append(str_i+"_lat_x")
        columns_name_long_x.append(str_i+"_long_x")
        columns_name_lat_y.append(str_i+"_lat_y")
        columns_name_long_y.append(str_i+"_long_y")
        columns_name_center_dist_x.append(str_i+"_center_dist_x")
        columns_name_center_dist_y.append(str_i+"_center_dist_y")

        lat_diff = (df_X[str_i+"_lat_x"].values - df_X[str_i+"_lat_y"].values)*lat_length
        long_diff = (df_X[str_i+"_long_x"].values - df_X[str_i+"_long_y"].values)*long_length
        dist = np.sqrt(lat_diff**2 + long_diff**2)
        df_X[str_i+"_dist"] = dist
        df_X[str_i+'_intersect'] = ((df_X[str_i+'_lat_max_x'] >= df_X[str_i+'_lat_min_y']) &
                              (df_X[str_i+'_lat_max_y'] >= df_X[str_i+'_lat_min_x']) &
                              (df_X[str_i+'_long_max_x'] >= df_X[str_i+'_long_min_y']) &
                              (df_X[str_i+'_long_max_y'] >= df_X[str_i+'_long_min_x'])) * 1
        center_x_lat_diff = (df_X[str_i+"_lat_x"].values - CENTER_LAT)*lat_length
        center_y_lat_diff = (df_X[str_i+"_lat_y"].values - CENTER_LAT)*lat_length
        
        center_x_long_diff = (df_X[str_i+"_long_x"].values - CENTER_LONG)*long_length
        center_y_long_diff = (df_X[str_i+"_long_y"].values - CENTER_LONG)*long_length

        df_X[str_i+"_center_dist_x"] = np.sqrt(center_x_lat_diff**2 + center_x_long_diff**2)
        df_X[str_i+"_center_dist_y"] = np.sqrt(center_y_lat_diff**2 + center_y_long_diff**2)
    
    # считаю среднее расстояние    
    df_X['dist_mean'] = df_X[columns_name_dist].mean(axis = 1)
    df_X['intersect_mean'] = df_X[columns_name_intersect].mean(axis = 1)
    df_X['intersect_count'] = df_X[columns_name_dist].count(axis = 1)/len(bins_list)
    df_X['center_dist_x_mean'] = df_X[columns_name_center_dist_x].mean(axis = 1)
    df_X['center_dist_y_mean'] = df_X[columns_name_center_dist_y].mean(axis = 1)
    
    if(dist_ind):
        columns_name = [*columns_name, *columns_name_dist]
    
    if(coord_ind):
        columns_name = [*columns_name, *columns_name_lat_x, *columns_name_lat_y, *columns_name_long_x, *columns_name_long_y]

    if(center_dist_ind):
        columns_name = [*columns_name, *columns_name_center_dist_x, *columns_name_center_dist_y]
    
    columns_name = [*columns_name, 
                    *['dist_mean', 'intersect_mean', 'intersect_count', 'center_dist_x_mean', 'center_dist_y_mean']]
        
    columns_name = [*columns_name, *['is_pair', 'msisdn_x', 'msisdn_y']]
    df_X = df_X[columns_name].set_index(['is_pair', 'msisdn_x', 'msisdn_y'])
    df_X.columns = df_X.columns + '_' + identifier
    
    print(df_X.shape)
    
    return df_X

In [35]:
def ProcessPhoneFeaturesPivot(df_msisdn_device, msisdn_pairs, df):
    
    msisdn_imei_count = df[~df['imei'].isna()].groupby(['msisdn', 'imei'])['lac'].count().reset_index().sort_values('lac', ascending = False)
    pairs_with_same_imei = msisdn_imei_count.merge(msisdn_imei_count, on = ['imei'])
    pairs_with_same_imei = pairs_with_same_imei[['msisdn_x', 'msisdn_y']]
    pairs_with_same_imei = pairs_with_same_imei.drop_duplicates()
    pairs_with_same_imei = pairs_with_same_imei[pairs_with_same_imei['msisdn_x'] != pairs_with_same_imei['msisdn_y']]
    pairs_with_same_imei['same_imei'] = 1

    df_X = msisdn_pairs.merge(df_msisdn_device.rename(columns = {'msisdn':'msisdn_x'}), on = ['msisdn_x']).merge(df_msisdn_device.rename(columns = {'msisdn':'msisdn_y'}), on = ['msisdn_y'])

    df_X = df_X.merge(pairs_with_same_imei, on = ['msisdn_x', 'msisdn_y'], how = 'left')
    df_X['same_imei'] = df_X['same_imei'].fillna(0)
    df_X.set_index(['is_pair', 'msisdn_x', 'msisdn_y'], inplace = True)
    
    return df_X[['imei_x', 'type_x', 'vendor_x', 'platform_x', 'imei_y', 'type_y', 'vendor_y', 'platform_y', 'same_imei']]